## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Karratha,AU,-20.7377,116.8463,80.51,47,74,9.66,broken clouds
1,1,Albany,US,42.6001,-73.9662,40.32,54,3,3.51,clear sky
2,2,Chapleau,CA,47.8405,-83.4034,37.49,81,90,4.61,overcast clouds
3,3,Khani,GE,41.9563,42.9566,38.77,95,97,1.50,overcast clouds
4,4,Rikitea,PF,-23.1203,-134.9692,76.59,78,18,15.12,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Rikitea,PF,-23.1203,-134.9692,76.59,78,18,15.12,few clouds
10,10,Quatre Cocos,MU,-20.2078,57.7625,76.08,76,18,9.10,few clouds
13,13,Moerai,PF,-22.4333,-151.3333,74.79,73,100,8.01,overcast clouds
14,14,Carnarvon,AU,-24.8667,113.6333,62.67,77,0,9.22,clear sky
15,15,Castro,BR,-24.7911,-50.0119,73.92,65,60,5.95,broken clouds
23,23,Kang,BW,-23.6752,22.7876,74.82,25,14,7.34,few clouds
29,29,Souillac,MU,-20.5167,57.5167,76.35,83,40,4.61,scattered clouds
32,32,Mahebourg,MU,-20.4081,57.7000,76.24,83,40,4.61,scattered clouds
33,33,Lagoa,PT,39.0500,-27.9833,63.45,77,72,34.14,broken clouds
36,36,Bambanglipuro,ID,-7.9500,110.2833,77.76,89,98,3.40,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                6
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Rikitea,PF,-23.1203,-134.9692,76.59,78,18,15.12,few clouds
10,10,Quatre Cocos,MU,-20.2078,57.7625,76.08,76,18,9.10,few clouds
13,13,Moerai,PF,-22.4333,-151.3333,74.79,73,100,8.01,overcast clouds
14,14,Carnarvon,AU,-24.8667,113.6333,62.67,77,0,9.22,clear sky
15,15,Castro,BR,-24.7911,-50.0119,73.92,65,60,5.95,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Rikitea,PF,76.59,few clouds,-23.1203,-134.9692,
10,Quatre Cocos,MU,76.08,few clouds,-20.2078,57.7625,
13,Moerai,PF,74.79,overcast clouds,-22.4333,-151.3333,
14,Carnarvon,AU,62.67,clear sky,-24.8667,113.6333,
15,Castro,BR,73.92,broken clouds,-24.7911,-50.0119,
23,Kang,BW,74.82,few clouds,-23.6752,22.7876,
29,Souillac,MU,76.35,scattered clouds,-20.5167,57.5167,
32,Mahebourg,MU,76.24,scattered clouds,-20.4081,57.7000,
33,Lagoa,PT,63.45,broken clouds,39.0500,-27.9833,
36,Bambanglipuro,ID,77.76,overcast clouds,-7.9500,110.2833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.loc[(hotel_df["Hotel Name"] == "")] = np.nan
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Rikitea,PF,76.59,few clouds,-23.1203,-134.9692,People ThankYou
10,Quatre Cocos,MU,76.08,few clouds,-20.2078,57.7625,Sunrise Attitude Hotel
13,Moerai,PF,74.79,overcast clouds,-22.4333,-151.3333,Le Manotel
14,Carnarvon,AU,62.67,clear sky,-24.8667,113.6333,Hospitality Carnarvon
15,Castro,BR,73.92,broken clouds,-24.7911,-50.0119,CHACARA BAILLY
23,Kang,BW,74.82,few clouds,-23.6752,22.7876,Kang Ultra Stop
29,Souillac,MU,76.35,scattered clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
32,Mahebourg,MU,76.24,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
33,Lagoa,PT,63.45,broken clouds,39.0500,-27.9833,Casa Das Faias
36,Bambanglipuro,ID,77.76,overcast clouds,-7.9500,110.2833,Puri Brata


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))